# Script to auto-build all objects required for COA-sourcing of TCA ResUsage Raw tables

<br><br><br>
## Initial setup of environment

In [1]:
import pandas as pd
import teradatasql as tddb
from json import dumps
from decimal import Decimal
import getpass as gp

# define your credentials:
username = 'sh186014'
password = gp.getpass('Enter Transcend password')
host = 'tdprd.td.teradata.com'
host = 'tddevtest.td.teradata.com'


def execute(sql="", return_data=False, ignore_errors=[]):
    "executes sql against Transcend, returning no dataset"
    print("execution started: %s" %sql[:200])
    conn_str = dumps(dict(host=host, user=username, password=password, encryptdata='false', logmech='LDAP'))
    conn = tddb.connect(conn_str)
    with conn.cursor() as csr:
        csr.execute(sql, ignoreErrors=ignore_errors)
        print("execution complete")
        if return_data:
            columns = [d[0] for d in csr.description]
            data = csr.fetchall()
            print("rows: %d, columns: %s" %(csr.rowcount, columns))
            pd.set_option('display.max_rows', None)
            return pd.DataFrame(data=([float(c) if isinstance(c, Decimal) else c for c in row] for row in data), columns=columns)
    return None    


def substitute(text: str, replacedict: dict) -> str:
    for find,replace in replacedict.items():
        text = text.replace('{%s}' %find, replace)
    return text

# test
execute('select * from dbc.dbcinfo', True)

Enter Transcend password ············


execution started: select * from dbc.dbcinfo
execution complete
rows: 3, columns: ['InfoKey', 'InfoData']


,InfoKey,InfoData
0,VERSION,16.20.53.07
1,RELEASE,16.20.53.07
2,LANGUAGE SUPPORT MODE,Standard


<br><br><br>
## Define all SQL required

In [3]:
sqls = {}
sqls['drop stage']  = """\nDROP TABLE {db}.stg_tca_resusage{proc}_{version} ;"""
sqls['drop target'] = """\nDROP TABLE {db}.coa_tca_resusage{proc}_{version} ;"""
sqls['drop view']   = """\nDROP TABLE {db}.coa{vwsuffix}_tca_resusage{proc}_{version} ;"""

sqls['create stage'] = """
CREATE SET GLOBAL TEMPORARY TABLE {db}.stg_tca_resusage{proc}_{version}  as
(
 SELECT ''(varchar(128)) as Site_ID, a.*  
 FROM raw_wla_vw.resusage{proc}_hst_{version} as a
) 
with no data
primary index (Site_ID, TheDate, NodeID)
on commit preserve rows;"""

sqls['drop stage columns'] =  "ALTER TABLE {db}.stg_tca_resusage{proc}_{version} DROP {column};"

sqls['create target'] = """
CREATE TABLE {tdb}.coa_tca_resusage{proc}_{version}
as {sdb}.stg_tca_resusage{proc}_{version}
with no data ; """

sqls['add target columns'] = "ALTER TABLE  {db}.coa_tca_resusage{proc}_{version} ADD {name} {type} ;"

sqls['replace view'] = """
replace view  {vdb}.coa_tca_resusage{proc}_{version}{vwsuffix}  as locking row for access
Select * from {tdb}.coa_tca_resusage{proc}_{version};"""


sqls['stored proc'] = """REPLACE PROCEDURE {tdb}.sp_tca_resusage{proc}_{version}() 
SQL SECURITY CREATOR
BEGIN     /* this is a simplified example, excluding any audit logging */ 

    DECLARE proc_name CHAR(30);  
    DECLARE batch_nbr INTEGER ;  
    
    /* someone will need to fill these values from ETL */
    SET proc_name = 'Unknown';
    SET batch_nbr = 0        ;   
   
    /* clear pre-existing records per site, date, and hour */
    Delete from {tdb}.coa_tca_resusage{proc}_{version}
    where (Site_ID, TheDate, TheTime) in
    (Select Site_ID, TheDate, TheTime from {sdb}.stg_tca_resusage{proc}_{version});
    
    /* insert new data */
    INSERT INTO {tdb}.coa_tca_resusage{proc}_{version}
    Select a.*
    , Current_Timestamp(6)  as di_created_ts 
    , NULL                  as di_upd_ts     
    , 'COA'                 as di_data_src_cd
    , :proc_name            as di_proc_name  
    , :batch_nbr            as di_batch_nbr  
    from {sdb}.stg_tca_resusage{proc}_{version} as a;
    
    /* clear GTT stage table so rows aren't inserted twice */
    delete from {sdb}.stg_tca_resusage{proc}_{version} ;
        
END;"""

<br><br><br>
## Define deployment configuration -- PROD / DATA LAB

In [8]:
# ON PROD / DATA LAB:
vwsuffix = '_vw'
db = {'stg':'adlste_coa_stg', 'tbl':'adlste_coa', 'vw':'adlste_coa'}
allversions = ['1410','1500','1510','1600','1610','1620','1700','1705','1710']
coldrop = ['tca_system_id','di_batch_nbr','di_proc_name','di_create_ts','di_update_ts']
coladd  = [{'name':'di_created_ts', 'type':'TIMESTAMP(6)   DEFAULT CURRENT_TIMESTAMP(6)'}
          ,{'name':'di_upd_ts',     'type':'TIMESTAMP(6)'}
          ,{'name':'di_data_src_cd','type':'VARCHAR(4)'}
          ,{'name':'di_proc_name',  'type':'CHAR(30)'}
          ,{'name':'di_batch_nbr',  'type':'INTEGER'}]
process = { 'SPMA':  {'versions':allversions}
           ,'SPDSK': {'versions':allversions}
           ,'SVPR':  {'versions':allversions}
          }

host = 'tdprd.td.teradata.com'           

<br><br><br>
## Define deployment configuration -- DEV / t10cld

In [28]:
# ON DEV / tddevtest :
vwsuffix = ''
db = {'stg':'APP_TCA_TMP', 'tbl':'APP_TCA_TBL', 'vw':'APP_TCA_VW'}
allversions = ['1410','1500','1510','1600','1610','1620','1700','1705','1710']
coldrop = ['tca_system_id','di_batch_nbr','di_proc_name','di_create_ts','di_update_ts']
coladd  = [{'name':'di_created_ts', 'type':'TIMESTAMP(6)   DEFAULT CURRENT_TIMESTAMP(6)'}
          ,{'name':'di_upd_ts',     'type':'TIMESTAMP(6)'}
          ,{'name':'di_data_src_cd','type':'VARCHAR(4)'}
          ,{'name':'di_proc_name',  'type':'CHAR(30)'}
          ,{'name':'di_batch_nbr',  'type':'INTEGER'}]
process = { 'SPMA':  {'versions':allversions}
           ,'SPDSK': {'versions':allversions}
           ,'SVPR':  {'versions':allversions}
          }

host = 'tddevtest.td.teradata.com'            

Enter Transcend password ············


connect string: %s {"host": "t10cld.td.teradata.com", "user": "sh186014", "password": "*****", "encryptdata": "false", "logmech": "LDAP"}

STARTING:
------------------------------
select * from dbc.dbcinfo
rows: 3, columns: ['InfoKey', 'InfoData']


<br><br><br>
## Execute!
Logs below

In [ ]:
# GO!!!   Build all Table/view structures:

for proc, detail in process.items():
    for version in detail['versions']:
        
        execute('\n\n----- NEW PROCESS for %s for VERSION %s ' %(proc, version))
        sql = substitute(sqls['drop stage'], dict(db=db['stg'], proc=proc, version=version))
        execute(sql, False, [3807]) # table not found
    
        sql = substitute(sqls['create stage'], dict(db=db['stg'], proc=proc, version=version))
        execute(sql)
        
        for col in coldrop:
            sql = substitute(sqls['drop stage columns'], dict(db=db['stg'], proc=proc, version=version, column=col))
            execute(sql)

        sql = substitute(sqls['drop target'], dict(db=db['tbl'], proc=proc, version=version))
        execute(sql, False, [3807]) # table not found
        
        sql = substitute(sqls['create target'], dict(tdb=db['tbl'], sdb=db['stg'], proc=proc, version=version))
        execute(sql)
            
        for col in coladd:
            sql = substitute(sqls['add target columns'], dict(db=db['tbl'], proc=proc, version=version, 
                                                              name=col['name'], type=col['type']))
            execute(sql)
            
        sql = substitute(sqls['replace view'], dict(tdb=db['tbl'], vdb=db['vw'], proc=proc, version=version, vwsuffix=vwsuffix))
        execute(sql)

        sql = substitute(sqls['stored proc'], dict(tdb=db['tbl'], sdb=db['stg'], proc=proc, version=version))
        execute(sql)

        

<br><br><br>
## Build SQL for COA process

In [4]:
# build all COA SQL files:
# ON DEV / t10cld :
host = 'tddevtest.td.teradata.com'  

# define what tables/versions we're going after:
versions = {'1410':{'svpr':{}, 'spma':{}, 'spdsk':{}},
            '1500':{'svpr':{}, 'spma':{}, 'spdsk':{}},
            '1510':{'svpr':{}, 'spma':{}, 'spdsk':{}},
            '1600':{'svpr':{}, 'spma':{}, 'spdsk':{}},
            '1610':{'svpr':{}, 'spma':{}, 'spdsk':{}},
            '1620':{'svpr':{}, 'spma':{}, 'spdsk':{}},
            '1700':{'svpr':{}, 'spma':{}, 'spdsk':{}},
            '1705':{'svpr':{}, 'spma':{}, 'spdsk':{}},
            '1710':{'svpr':{}, 'spma':{}, 'spdsk':{}} }

# thin for testing
versions = {'1600':{'svpr':{}, 'spma':{}, 'spdsk':{}},
            '1610':{'svpr':{}, 'spma':{}, 'spdsk':{}},
            '1620':{'svpr':{}, 'spma':{}, 'spdsk':{}},
            '1700':{'svpr':{}, 'spma':{}, 'spdsk':{}},
            '1705':{'svpr':{}, 'spma':{}, 'spdsk':{}},
            '1710':{'svpr':{}, 'spma':{}, 'spdsk':{}} }

# SPMA
versions['1600']['spma']['groupby'] = {'Site_ID':"'{siteid}'", '*':'*'}
versions['1610']['spma']['groupby'] = {'Site_ID':"'{siteid}'", '*':'*'}
versions['1620']['spma']['groupby'] = {'Site_ID':"'{siteid}'", '*':'*'}
versions['1700']['spma']['groupby'] = {'Site_ID':"'{siteid}'", '*':'*'}
versions['1705']['spma']['groupby'] = {'Site_ID':"'{siteid}'", '*':'*'}
versions['1710']['spma']['groupby'] = {'Site_ID':"'{siteid}'", '*':'*'}

versions['1600']['spma']['keepers'] = ['']
versions['1610']['spma']['keepers'] = ['']
versions['1620']['spma']['keepers'] = ['']
versions['1700']['spma']['keepers'] = ['']
versions['1705']['spma']['keepers'] = ['']
versions['1710']['spma']['keepers'] = ['']


# SPDSK
versions['1600']['spdsk']['groupby'] = {'Site_ID':"'{siteid}'", '*':'*'}
versions['1610']['spdsk']['groupby'] = {'Site_ID':"'{siteid}'", '*':'*'}
versions['1620']['spdsk']['groupby'] = {'Site_ID':"'{siteid}'", '*':'*'}
versions['1700']['spdsk']['groupby'] = {'Site_ID':"'{siteid}'", '*':'*'}
versions['1705']['spdsk']['groupby'] = {'Site_ID':"'{siteid}'", '*':'*'}
versions['1710']['spdsk']['groupby'] = {'Site_ID':"'{siteid}'", '*':'*'}
versions['1600']['spdsk']['keepers'] = ['']
versions['1610']['spdsk']['keepers'] = ['']
versions['1620']['spdsk']['keepers'] = ['']
versions['1700']['spdsk']['keepers'] = ['']
versions['1705']['spdsk']['keepers'] = ['']
versions['1710']['spdsk']['keepers'] = ['']


# SVPR
versions['1600']['svpr']['groupby'] = {'Site_ID':"'{siteid}'",'TheDate':'TheDate','NodeID':'NodeID','TheTime':'TheTime',
           'VprType':"case when VprType like 'PE%' then 'PE' when VprType like 'GTW%' then 'GTW' when VprType like 'AMP%' then 'AMP' else 'OTHER' end"}
versions['1610']['svpr']['groupby'] = versions['1600']['svpr']['groupby']
versions['1620']['svpr']['groupby'] = versions['1600']['svpr']['groupby']
versions['1700']['svpr']['groupby'] = versions['1600']['svpr']['groupby']
versions['1705']['svpr']['groupby'] = versions['1600']['svpr']['groupby']
versions['1710']['svpr']['groupby'] = versions['1600']['svpr']['groupby']

versions['1600']['svpr']['keepers'] = ['FilePDbAcqs','FilePCiAcqs','FileSDbAcqs','FileSCiAcqs','FilePDbAcqKB','FilePCiAcqKB','FileSDbAcqKB','FileSCiAcqKB',
           'FilePDbAcqReads','FilePCiAcqReads','FilePDbPreReads','FilePCiPreReads','FileSDbAcqReads','FileSCiAcqReads','FileSDbPreReads',
           'FileSCiPreReads','FilePDbAcqReadKB ','FilePCiAcqReadKB','FilePDbPreReadKB ','FilePCiPreReadKB','FileSDbAcqReadKB ',
           'FileSCiAcqReadKB','FileSDbPreReadKB ','FileSCiPreReadKB','FilePDbFWrites ','FilePCiFWrites','FileSDbFWrites ','FileSCiFWrites',
           'FilePDbFWriteKB ','FilePCiFWriteKB','FileSDbFWriteKB ','FileSCiFWriteKB','FilePDbDyRRels','FilePDbCnRRels','FilePDbDyAWrites',
           'FilePDbDyRRelKB','FilePDbCnRRelKB','FilePDbDyAWriteKB','FileSDbDyRRels','FileSDbCnRRels','FileSDbDyRRelKB','FileSDbCnRRelKB',
           'FileSDbDyAWriteKB','FileTJtFWriteKB','FileTJtDyAWriteKB','FileTJtPreReadKB','FileTJtAcqReadKB','FilePreCompMB','FilePostCompMB',
           'FilePreUnCompMB','FilePostUnCompMB','FileCompDBs','FileUnCompDBs','FileCompCPU','FileUnCompCPU','FileFcrRequests','FileFcrDeniedUser',
           'FileFcrDeniedKern','FileFcrBlocksRead','FileFcrDeniedThreshKern ','FileFcrDeniedThreshUser','FileFcrDeniedCache','CPUUExecPart13',
           'CPUUServPart13','CPUUExecPart14','CPUUServPart14','CPUUExecPart12','CPUUServPart12','CPUUExecPart10','CPUUServPart10','CPUUExecPart11',
           'CPUUServPart11','VHAgedOut','VHAgedOutKB','VHLogicalDBRead','VHLogicalDBReadKB','VHPhysicalDBRead','VHPhysicalDBReadKB']
versions['1610']['svpr']['keepers'] = versions['1600']['svpr']['keepers']
versions['1620']['svpr']['keepers'] = versions['1600']['svpr']['keepers']
versions['1700']['svpr']['keepers'] = versions['1600']['svpr']['keepers']
versions['1705']['svpr']['keepers'] = versions['1600']['svpr']['keepers']
versions['1710']['svpr']['keepers'] = versions['1600']['svpr']['keepers']

print('done!')

done!


In [5]:
for version, tables in versions.items():
    for table, settings in tables.items():
        
        hdr = 'TCA Process for ResUsage%s == %s' %(table.upper(), version)
        print('\n%s\n%s\n%s\n' %('='*36, hdr, '='*36))
        
        # pull the right structure from existing DBC definitions
        sql = """select trim(ColumnName) as ColumnName, ColumnType from dbc.ColumnsV
             where DatabaseName = 'DEV_APP_TCA_TMP' and TableName = 'stg_tca_resusage%s_%s'
             order by ColumnID""" %(table, version)
        df = execute(sql, True)
        
        groupby = [x.lower() for x in list(settings['groupby'].keys())]
        keepers = [x.lower() for x in settings['keepers']]
        
        print(groupby)
        
         # build final output SQL:
        filename = 'tca_resusage%s.%sxx.coa.sql' %(table.upper(), int(int(version)/100) )
        print('====> SAVE TO FILE: %s=\n' %filename)
        sql = []
        sql.append('/* %s */ ' %hdr)
        sql.append('/*  Parameters:')
        sql.append('     startdate = {startdate}')
        sql.append('     enddate = {enddate}')
        sql.append('     resusage%s = {resusage%s}' %(table, table))
        sql.append('     dbprefix = {dbprefix} <-- probably blank, but set to "Dev_" in the FileSet during TCA Testing')
        sql.append('     dbsversion_label = {dbsversion_label}')
        sql.append(' */')
        sql.append(' ')
        
        sql.append('/*{{save:tca_resusage%s_{dbsversion_label}.csv}}*/' %table.upper())
        sql.append('/*{{load:{dbprefix}APP_TCA_TMP.stg_tca_resusage%s_{dbsversion_label}}}*/' %table.upper())
        sql.append('/*{{call:{dbprefix}APP_TCA_TBL.sp_tca_resusage%s_{dbsversion_label}()}}*/' %table.upper())
        
        
        sql.append('Select ')
        delim='  '
        for index, row in df.iterrows():

            if row['ColumnName'].lower() in groupby:  
                sql.append(delim + settings['groupby'][row['ColumnName']] + ' as ' + row['ColumnName'])
                
            elif row['ColumnName'].lower() in keepers:
                sql.append(delim + 'sum(' + row['ColumnName'] + ') as ' + row['ColumnName'])

            elif '*' in groupby:  
                sql.append(delim + row['ColumnName'])

            else:
                sql.append(delim + 'NULL as ' + row['ColumnName'])
            delim = ' ,'
        sql.append('from {resusage%s} -- pdcrinfo.resusage%s_hst or dbc.resusage%s' %(table,table.upper(),table.upper()))
        sql.append('where TheDate between {startdate} and {enddate}')
        
        if '*' not in settings['groupby'].keys():
            sql.append('group by ')
            delim='  '
            for grp in list(settings['groupby'].values()):
                sql.append(delim + grp)
                delim = ' ,'

        sql.append(';')
        output = '\n'.join(sql)
        # print(output)
        f = open("resusage_output/%s" %filename, "w")
        f.write(output)
        f.close()
        
        
print('Done!')


TCA Process for ResUsageSVPR == 1600

execution started: select trim(ColumnName) as ColumnName, ColumnType from dbc.ColumnsV
             where DatabaseName = 'DEV_APP_TCA_TMP' and TableName = 'stg_tca_resusagesvpr_1600'
             order by ColumnID
execution complete
rows: 452, columns: ['ColumnName', 'ColumnType']
['site_id', 'thedate', 'nodeid', 'thetime', 'vprtype']
====> SAVE TO FILE: tca_resusageSVPR.16xx.coa.sql=


TCA Process for ResUsageSPMA == 1600

execution started: select trim(ColumnName) as ColumnName, ColumnType from dbc.ColumnsV
             where DatabaseName = 'DEV_APP_TCA_TMP' and TableName = 'stg_tca_resusagespma_1600'
             order by ColumnID
execution complete
rows: 260, columns: ['ColumnName', 'ColumnType']
['site_id', '*']
====> SAVE TO FILE: tca_resusageSPMA.16xx.coa.sql=


TCA Process for ResUsageSPDSK == 1600

execution started: select trim(ColumnName) as ColumnName, ColumnType from dbc.ColumnsV
             where DatabaseName = 'DEV_APP_TCA_TMP' and

In [6]:


# pull definition from the final GTT landing zone, to make sure they match
for proc, settings in processes.items():
    
    for version in settings['versions']:
        # pull the right structure from existing DBC definitions
        sql = """select trim(ColumnName) as ColumnName, ColumnType from dbc.ColumnsV
             where DatabaseName = 'APP_TCA_TMP' and TableName = 'stg_tca_resusageSVPR_%s'
             order by ColumnID""" %version
        df = execute(sql, True)
        
        groupby = settings['groupby']
    
        # build final output SQL:
        sql = ['Select ']
        delim='  '
        for index, row in df.iterrows():

            if row['ColumnName'] in list(settings['groupby'].keys()):  
                sql.append(delim + settings['groupby'][row['ColumnName']])

            elif row['ColumnName'].lower() in settings['keepers']:
                sql.append(delim + row['ColumnName'])

            else:
                sql.append(delim + 'NULL as ' + row['ColumnName'])
            delim = ' ,'
        sql.append('from pdcrinfo.resusageSVPR_hst')
        sql.append('where TheDate between DATE-2 and DATE-1')

        print('\n'.join(sql))
    
    
    

{'Site_ID': '{siteid} as Site_ID',
 'TheDate': 'TheDate',
 'NodeID': 'NodeID',
 'TheTime': 'TheTime',
 'VprType': "case when VprType like 'PE%' then 'PE' when VprType like 'GTW%' then 'GTW' when VprType like 'AMP%' then 'AMP' else 'OTHER' end as VprType"}

In [35]:
list(settings['groupby'].values())

['{siteid}',
 'TheDate',
 'NodeID',
 'TheTime',
 "case when VprType like 'PE%' then 'PE' when VprType like 'GTW%' then 'GTW' when VprType like 'AMP%' then 'AMP' else 'OTHER' end"]